#### Imports

In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
nltk.download('vader_lexicon')
from scipy.special import softmax
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, XLNetTokenizer, TFXLNetForSequenceClassification

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


## Filtering 3-Star Reviews

In [3]:
# Load the combined reviews data from the saved file
combined_reviews_df = pd.read_csv('processed_data/combined_reviews.csv', encoding='utf-8', engine='python')

# Filter for reviews with rating = 3 to perform sentiment analysis on the same
three_stars = combined_reviews_df[combined_reviews_df['rating'] == 3]

#print(filtered_df.head())
print(f"Total reviews with rating of 3: {len(three_stars)}")

Total reviews with rating of 3: 60703




---



# Sentiment Analysis on Product Reviews

## xlnet-base-cased-product-review-sentiment-analysis

Below we use [xlnet-base-cased-product-review-sentiment-analysis](https://huggingface.co/dipawidia/xlnet-base-cased-product-review-sentiment-analysis)model from Hugging face to perform sentiment analysis.

In [4]:
# Initialize the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")
model = TFXLNetForSequenceClassification.from_pretrained("dipawidia/xlnet-base-cased-product-review-sentiment-analysis")

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)
    preds = model.predict(dict(tokenize_text))['logits']
    class_preds = np.argmax(tf.keras.layers.Softmax()(preds), axis=-1)
    labels = ['Positive' if pred == 1 else 'Negative' for pred in class_preds]
    return labels

# Batch size for processing reviews
batch_size = 64

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive and negative sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

...

Processing reviews: 100%|█████████▉| 948/949 [1:11:54<00:08,  8.01s/batch]

1/1 [==============================] - 3s 3s/step


Processing reviews: 100%|██████████| 949/949 [1:11:58<00:00,  4.55s/batch]

sentiment
Negative    38999
Positive    21704
Name: count, dtype: int64



<ipython-input-4-71cf0fc2eb74>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = sentiment_results


In [5]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

Positive Sentiment Examples:
                                              review_text
14      It’s super glossy and looks amazing but doesn’...
40      This product is marketed as a sleeping mask bu...
53      I think this is overpriced for what it is - li...
92      Meh, I mean it’s ok. I like the Fenty skin one...
112     I used to love this product. Because it feels ...
...                                                   ...
740237  is alright nothing special leaves skin soft ea...
740251  Saw all the AB hype and decided to give the br...
740255  Very moisturizing and feels really nice, but i...
740289  Pros:-soothing and moisturizing - it feels lik...
740355  Been using it for a month but sadly didn’t any...

[21704 rows x 1 columns]


In [6]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])

Negative Sentiment Examples:
                                              review_text
11      Does the gummy bear mask smell absolutely deli...
30      ive tried my friend’s a few times and it does ...
60      I have a sample size of this product. It is ve...
64      I don’t see the hype for this product. It does...
80      I tried it first from the mini version Sephora...
...                                                   ...
740268  This is not the best or the worst makeup remov...
740272  This is a thick moisturizing cream, and compar...
740276  So so. Sticky, smells weird. Not much help to ...
740277  I paid a fortune for this cream but I’m not co...
740354  Im heading into my 30s so I wanted to splurge ...

[38999 rows x 1 columns]


## Get all positive and negative reviews


In [7]:
positive_reviews = three_stars[three_stars['sentiment'] == 'Positive']
negative_reviews = three_stars[three_stars['sentiment'] == 'Negative']

positive_path = 'processed_data/1.positive_reviews.csv'
positive_reviews.to_csv(positive_path, index=False)

negative_path = 'processed_data/1.negative_reviews.csv'
negative_reviews.to_csv(negative_path, index=False)



---



## SentimentIntensityAnalyzer

We also tried to compute sentiment analysis with [SentimentIntensityAnalyzer](https://www.nltk.org/howto/sentiment.html) - one of the most useful NLTK (Natural Language Toolkit) features. It is a pre-trained model that can be used to perform sentiment analysis on text data.

In [8]:
sia = SentimentIntensityAnalyzer()

def get_sentiment(row):
    # Get the sentiment scores
    scores = sia.polarity_scores(row['review_text'])

    # Decide sentiment based on compound score
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

three_stars['sentiment'] = three_stars.apply(get_sentiment, axis=1)

# Count positives and negatives
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

sentiment
Positive    49700
Negative     8449
Neutral      2554
Name: count, dtype: int64


<ipython-input-8-52efdc083d75>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = three_stars.apply(get_sentiment, axis=1)


## Get all positive and negative reviews


In [9]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

Positive Sentiment Examples:
                                              review_text
11      Does the gummy bear mask smell absolutely deli...
14      It’s super glossy and looks amazing but doesn’...
30      ive tried my friend’s a few times and it does ...
40      This product is marketed as a sleeping mask bu...
53      I think this is overpriced for what it is - li...
...                                                   ...
740272  This is a thick moisturizing cream, and compar...
740277  I paid a fortune for this cream but I’m not co...
740289  Pros:-soothing and moisturizing - it feels lik...
740354  Im heading into my 30s so I wanted to splurge ...
740355  Been using it for a month but sadly didn’t any...

[49700 rows x 1 columns]


In [10]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])

Negative Sentiment Examples:
                                              review_text
113     I would not repurchase as there are better one...
122     I like this product overall, I find that moist...
277     I have mixed feelings about the product. Yes i...
493     the main reason i bought this was for the scen...
679     Had to try Mango, but it’s scent/ flavour is t...
...                                                   ...
739868  Smells like vinegar and dries a little sticky....
739875  it did nothing for my skin, just an extra step...
739986  Of all the Origins skincare products I own, th...
740105  This was super foamy which was nice. It did dr...
740251  Saw all the AB hype and decided to give the br...

[8449 rows x 1 columns]


In [11]:
positive_reviews = three_stars[three_stars['sentiment'] == 'Positive']
negative_reviews = three_stars[three_stars['sentiment'] == 'Negative']

positive_path = 'processed_data/2.positive_reviews.csv'
positive_reviews.to_csv(positive_path, index=False)

negative_path = 'processed_data/2.negative_reviews.csv'
negative_reviews.to_csv(negative_path, index=False)



---



## cardiffnlp/twitter-xlm-roberta-base-sentiment

In [12]:
# Model path for the new model
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = TFAutoModelForSequenceClassification.from_pretrained(model_path)

# Function to analyze sentiment for a batch of reviews
def get_sentiment_batch(reviews_batch):
    # Tokenize the batch of reviews
    tokenize_text = tokenizer(reviews_batch.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=512)

    # Get predictions from the model
    preds = model(**tokenize_text)['logits']

    # Apply softmax to get class probabilities
    scores = softmax(preds.numpy(), axis=-1)

    # Get the class with the highest probability (0 = negative, 2 = positive)
    class_preds = np.argmax(scores, axis=-1)

    # Assign sentiment labels based on the class prediction
    labels = ['Negative' if pred == 0 else 'Neutral' if pred == 1 else 'Positive' for pred in class_preds]

    return labels

# Batch size for processing reviews
batch_size = 64

# Ensure the reviews are valid strings and filter out NaN values
three_stars['review_text'] = three_stars['review_text'].fillna('').astype(str)

# Initialize the progress bar for batching
num_batches = len(three_stars) // batch_size + (1 if len(three_stars) % batch_size != 0 else 0)

# List to hold sentiment results
sentiment_results = []

# Process reviews in batches
for i in tqdm(range(0, len(three_stars), batch_size), total=num_batches, desc="Processing reviews", unit="batch"):
    batch = three_stars['review_text'][i:i + batch_size]  # Get the current batch of reviews
    batch_sentiments = get_sentiment_batch(batch)  # Get sentiments for the current batch
    sentiment_results.extend(batch_sentiments)  # Append results

# Assign the sentiment results back to the dataframe
three_stars['sentiment'] = sentiment_results

# Count positive, negative, and neutral sentiments
sentiment_counts = three_stars['sentiment'].value_counts()
print(sentiment_counts)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.
<ipython-input-12-df9068febab8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['review_text'] = three_stars['review_text'].fillna('').astype(str)
Processing reviews: 100%|██████████| 949/949 [20:56<00:00,  1.32s/batch]

sentiment
Negative    34045
Positive    21846
Neutral      4812
Name: count, dtype: int64



<ipython-input-12-df9068febab8>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three_stars['sentiment'] = sentiment_results


## Get all positive and negative reviews


In [13]:
# Display a few positive sentiment examples
positive_sentiments = three_stars[three_stars['sentiment'] == 'Positive']
print("Positive Sentiment Examples:")
print(positive_sentiments[['review_text']])

Positive Sentiment Examples:
                                              review_text
11      Does the gummy bear mask smell absolutely deli...
30      ive tried my friend’s a few times and it does ...
40      This product is marketed as a sleeping mask bu...
53      I think this is overpriced for what it is - li...
80      I tried it first from the mini version Sephora...
...                                                   ...
740251  Saw all the AB hype and decided to give the br...
740255  Very moisturizing and feels really nice, but i...
740267  Feels amazing on the skin, absorbed quickly an...
740277  I paid a fortune for this cream but I’m not co...
740289  Pros:-soothing and moisturizing - it feels lik...

[21846 rows x 1 columns]


In [14]:
# Display a few negative sentiment examples
negative_sentiments = three_stars[three_stars['sentiment'] == 'Negative']
print("Negative Sentiment Examples:")
print(negative_sentiments[['review_text']])

Negative Sentiment Examples:
                                              review_text
14      It’s super glossy and looks amazing but doesn’...
60      I have a sample size of this product. It is ve...
64      I don’t see the hype for this product. It does...
82      I really wanted to love this, but I ended up j...
113     I would not repurchase as there are better one...
...                                                   ...
740268  This is not the best or the worst makeup remov...
740272  This is a thick moisturizing cream, and compar...
740276  So so. Sticky, smells weird. Not much help to ...
740354  Im heading into my 30s so I wanted to splurge ...
740355  Been using it for a month but sadly didn’t any...

[34045 rows x 1 columns]


In [15]:
positive_reviews = three_stars[three_stars['sentiment'] == 'Positive']
negative_reviews = three_stars[three_stars['sentiment'] == 'Negative']

positive_path = 'processed_data/3.positive_reviews.csv'
positive_reviews.to_csv(positive_path, index=False)

negative_path = 'processed_data/3.negative_reviews.csv'
negative_reviews.to_csv(negative_path, index=False)